In [18]:
import numpy as np
import math
from os import path
from sklearn.feature_extraction import image
import imageio

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

def convertCSVtoNP(Filepath):
    supportedFileFormats = ["png","jpeg","jpg","bmp"]

    if not path.isfile(Filepath):
        raise ValueError("Path: '{}' is no file!".format(Filepath))

    if not path.splitext(Filepath)[1] in [".txt",".csv"]:
        raise ValueError("Selected fileformat {} is not valid. Only .csv or .txt are allowed!".format(path.splitext(Filepath)[1]))

    with open(Filepath, "r") as f:
        #read all the lines from the file 
        lines = f.readlines()

        #first line only has comments, we ignore it 

        #second line has the image dimensions so we need to read it as integer 
        try: 
            dataWidth = int(lines[1])
            x,y = [(int(x)) for x in lines[2].split()]
        except ValueError:
            raise ValueError("Error reading the image dimensions. Check that there are two integers seperated by whitespace!")

        #create an empty numpy array to store the image data.
        outputArray = np.zeros((x,y,dataWidth))

        #drop the first two rows of data
        lines = lines[3:]

        for index, line in enumerate(lines):
            #each line contains the r,g,b information separated by blank spaces  
            try:
                inputValues = [(int(x)) for x in line.split()]
            except ValueError:
                raise ValueError("Error in line {}. Check that there are three integers seperated by whitespace!".format(index+2))

            outputArray[math.floor(index / y),index % y] = np.array(inputValues)

    return outputArray

def loadNetworkModel(Path, MaxValue = 2.0):
    #as we are using an advanced activation function in the original file we have to create it here as well. Otherwise Tensorflow does 
    #not know what function is refferenced 
    def create_relu_advanced(max_value=2.0):        
            def relu_advanced(x):
                return K.relu(x, max_value=K.cast_to_floatx(max_value))
            return relu_advanced

    #create a special softmax function so that we limit the output before applying softmax to -1...1
    #usefull for the calculation inside the neural network with fixedpoint. Otherwise we might oversaturate the output 
    def tanh_softmax(x):
        return K.softmax(K.tanh(x))          
        
    #as we want the second model to feed into the next model we need to change the activation function of the last layer
    model = keras.models.load_model(Path,custom_objects={'relu_advanced':create_relu_advanced(MaxValue),'tanh_softmax':tanh_softmax})

    return model

colors = np.array([(0,0,0),(0,255,0),(0,0,255),(255,255,0),(255,0,0),(0,255,255),(0,0,0),(255,255,255)])

In [3]:
#simOutput = convertCSVtoNP(r"C:\Users\Unknown\Documents\Master-Convolution\VHDL\Code\Testbench\out1.txt")
#simOutput = simOutput.astype(float) / 2**5

firstModel = loadNetworkModel(r"C:\Users\Unknown\Documents\Master-Convolution\Python\first.h5")
secondModel  = loadNetworkModel(r"C:\Users\Unknown\Documents\Master-Convolution\Python\second.h5")

In [4]:
originalImage = imageio.imread(r"C:\Users\Unknown\Documents\Master-Convolution\Python\Subshapes\input.jpg")
firstNetworkInput = image.extract_patches_2d(originalImage[:,:,0],(7,7))

firstNetworkInput -= np.min(firstNetworkInput)
firstNetworkInput = firstNetworkInput.astype(float) / np.max(firstNetworkInput).astype(float)

firstModelOutput = firstModel.predict(firstNetworkInput, batch_size=2048)

In [5]:
firstModelOutput = np.reshape(firstModelOutput,(originalImage.shape[0]-6,-1,7))
#simOutput = simOutput[:firstModelOutput.shape[0],:firstModelOutput.shape[1]]

In [6]:
#secondStageInputSim = image.extract_patches_2d(simOutput,(21,21))

secondStageInputPy  = image.extract_patches_2d(firstModelOutput,(21,21))
#secondStageInputSim = secondStageInputSim[:,::7,::7].copy()
secondStageInputPy  = secondStageInputPy[:,::7,::7].copy()

In [7]:
bigInput = image.extract_patches_2d(originalImage[:,:,0],(21,21))
bigInput -= np.min(bigInput)
bigInput = bigInput.astype(float) / np.max(bigInput).astype(float)

secondStageInputTraining = np.zeros((len(bigInput),7,3,3))

#in order to predict the output of the first network we need an array with the shape of (-1,firstInputShape[1],firstInputShape[2])
#so we need to subset the input. 
for j in range(3):
    for k in range(3):
        #create the subset from the patches  
        subSlice = bigInput[:,j* 7: (j+1)*7, k * 7: (k+1)*7]
        #apply the network to the patches
        secondStageInputTraining[:,:,j,k] = firstModel.predict(subSlice ,batch_size=65536)

In [8]:
#secondStageInputSim = np.transpose(secondStageInputSim, (0,3,1,2))
secondStageInputPy  = np.transpose(secondStageInputPy, (0,3,1,2))

In [9]:
#secondStageOutputSim        = secondModel.predict(secondStageInputSim,batch_size=2048)
secondStageOutputPy         = secondModel.predict(secondStageInputPy,batch_size=2048)
secondStageOutputTraining   = secondModel.predict(secondStageInputTraining,batch_size=2048)

In [10]:
#secondStageOutputSim        = np.reshape(secondStageOutputSim, (firstModelOutput.shape[0]-20,-1,6))
secondStageOutputPy         = np.reshape(secondStageOutputPy, (firstModelOutput.shape[0]-20,-1,6))  
secondStageOutputTraining   = np.reshape(secondStageOutputTraining, (originalImage.shape[0]-20,-1,6))

In [23]:
colors = np.array([(0,0,0),(0,255,0),(0,0,255),(255,255,0),(255,0,0),(0,0,0),(0,0,0),(255,255,255)])

#imageSim        = colors[np.argmax(secondStageOutputSim,axis=2)]
imagePy         = colors[np.argmax(secondStageOutputPy,axis=2)]
imageTraining   = colors[np.argmax(secondStageOutputTraining,axis=2)]
imageFirstNetwork    = colors[np.argmax(firstModelOutput,axis=2)]

In [24]:
#imageio.imwrite("Sim.jpg",imageSim)
imageio.imwrite("Py.jpg",imagePy)
imageio.imwrite("Training.jpg",imageTraining)
imageio.imwrite("First Output.jpg", imageFirstNetwork)

Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.


In [12]:
#dif = simOutput-firstModelOutput
# dif = dif**2

# print(np.max(dif))
# print(np.std(dif))

# difSum = np.sum(dif,axis=2)

# difSum -= np.min(difSum)
# difSum = difSum/np.max(difSum)

# difPic = np.round(difSum * 255)
# imageio.imwrite("Dif.jpg",difPic)

3.9999990463257404
0.1609769080103107


Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


In [36]:
#simSecondLayerInput = convertCSVtoNP(r"C:\Users\Unknown\Documents\Master-Convolution\VHDL\Code\Testbench\out3.txt")

In [37]:
# print(simSecondLayerInput[125,712])

# simSecondLayerInput = np.reshape(simSecondLayerInput,(-1, simSecondLayerInput.shape[-1]))
# simSecondLayerInput = np.reshape(simSecondLayerInput,(-1,3,3,7))
# simSecondLayerInput = np.transpose(simSecondLayerInput,(0,3,1,2))

# simSecondLayerInput = simSecondLayerInput.astype(float) / 2**5

[-32. -32. -32. -32. -32. -32.  32. -32. -32. -32. -32. -32. -32.  32.
 -32. -32. -32. -32. -32. -32.  32. -32. -32. -32. -32. -32. -32.  32.
 -32. -32. -32. -32. -32. -32.  32. -32. -32. -32. -32. -32. -32.  32.
 -32. -32. -32. -32. -32. -32.  32. -32. -32. -32. -32. -32. -32.  32.
 -32. -32. -32. -32. -32. -32.  32.]
[[[-1. -1. -1.]
  [-1. -1. -1.]
  [-1. -1. -1.]]

 [[-1. -1. -1.]
  [-1. -1. -1.]
  [-1. -1. -1.]]

 [[-1. -1. -1.]
  [-1. -1. -1.]
  [-1. -1. -1.]]

 [[-1. -1. -1.]
  [-1. -1. -1.]
  [-1. -1. -1.]]

 [[-1. -1. -1.]
  [-1. -1. -1.]
  [-1. -1. -1.]]

 [[-1. -1. -1.]
  [-1. -1. -1.]
  [-1. -1. -1.]]

 [[ 1.  1.  1.]
  [ 1.  1.  1.]
  [ 1.  1.  1.]]]


In [38]:
#simFinalOut = secondModel.predict(simSecondLayerInput, batch_size = 8096)

In [39]:
# secondStageOutputSim    = np.reshape(simFinalOut, (720,1280,6))
# imageSim                = colors[np.argmax(secondStageOutputSim,axis=2)]
# imageio.imwrite("SimTotal.jpg",imageSim)

Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.


In [13]:
bigInput = image.extract_patches_2d(originalImage[:,:,0],(21,21))
bigInput -= np.min(bigInput)
bigInput = bigInput.astype(float) / np.max(bigInput).astype(float)

secondStageInputTraining = np.zeros((len(bigInput),7,3,3))

#in order to predict the output of the first network we need an array with the shape of (-1,firstInputShape[1],firstInputShape[2])
#so we need to subset the input. 
for j in range(3):
    for k in range(3):
        missmatch = 7
        #create the subset from the patches  
        subSlice = bigInput[:,j* missmatch: (j*missmatch)+7, k * missmatch: (k*missmatch)+7]
        #apply the network to the patches
        secondStageInputTraining[:,:,j,k] = firstModel.predict(subSlice ,batch_size=65536)

secondStageOutputTraining   = secondModel.predict(secondStageInputTraining,batch_size=2048)
secondStageOutputTraining   = np.reshape(secondStageOutputTraining, (originalImage.shape[0]-20,-1,6))
imageTraining   = colors[np.argmax(secondStageOutputTraining,axis=2)]
imageio.imwrite("Training first.jpg",imageTraining)

Lossy conversion from int32 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.


In [48]:
# with open(r"C:\Users\Unknown\Documents\Master-Convolution\VHDL\Code\Testbench\in3.txt","w") as f:
#     f.write("First Layer Sim Out\n")
#     f.write("{} {}\n".format(secondStageOutputTraining.shape[0],secondStageOutputTraining.shape[1]))

#     for val in secondStageInputTraining:
#         for i in range(3*3*7):
#             f.write("{} ".format(int(np.transpose(val,(1,2,0)).flatten()[i]*2**5)))
#         f.write("\n")